# instalación de paquetes y verificación

In [27]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
import string
from datetime import datetime
from object_detection.utils import label_map_util
python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.7.9  which is compatible with this notebook


In [28]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 models/research/object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/tf1/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_featur

# Variables de entorno

In [35]:
training_dir = "training_masks"
images_dir  = "images_masks"
inference_graph_dir = "inference_graph_mask_rcnn_{}_{}_MVI_images".format(datetime.now().day,datetime.now().month)
label_map = label_map_util.load_labelmap(training_dir+"/label_map.pbtxt")
label_map_dict = label_map_util.get_label_map_dict(label_map)
n_classes=len(label_map_dict.keys())

In [30]:
!sed 's/n_classes/{n_classes}/' {training_dir}/mask_rcnn_inception_v2_coco.config >{training_dir}/temp.config
!sed -i 's/training_dir/{training_dir}/' {training_dir}/temp.config
!cat {training_dir}/temp.config

# Mask R-CNN with Inception V2
# Configured for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 4
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 800
        max_dimension: 1365
      }
    }
    number_of_stages: 3
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer 

# creación del dataset

In [36]:
# borramos las carpetas de train test y validación si es que existían de un entrenamiento anterior
!rm -rf {images_dir}/train {images_dir}/test {images_dir}/val

In [37]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test.
# el argumento -f indica el formato de las imágenes
!python3 scripts/train_test_split.py -i {images_dir} -o {images_dir} -tr 0.1 -vr 0.15 -f jpg
!tree -d {images_dir}

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data
images_masks
├── test
├── train
└── val

3 directories


In [38]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py {images_dir}/train {images_dir}/train/coco --labels {training_dir}/labels.txt
!python3 scripts/labelme2coco.py {images_dir}/test {images_dir}/test/coco --labels {training_dir}/labels.txt
!python3 scripts/labelme2coco.py {images_dir}/val {images_dir}/val/coco --labels {training_dir}/labels.txt
!tree -d {images_dir}

Creating dataset: images_masks/train/coco
Generating dataset from: images_masks/train/video6_frame10080.json
Generating dataset from: images_masks/train/video2_frame5220.json
Generating dataset from: images_masks/train/video1_frame8950.json
Generating dataset from: images_masks/train/video1_frame3950.json
Generating dataset from: images_masks/train/video2_frame5100.json
Generating dataset from: images_masks/train/video2_frame2100.json
Generating dataset from: images_masks/train/video1_frame4500.json
Generating dataset from: images_masks/train/video2_frame1080.json
Generating dataset from: images_masks/train/video2_frame6780.json
Generating dataset from: images_masks/train/video1_frame9350.json
Generating dataset from: images_masks/train/video2_frame2580.json
Generating dataset from: images_masks/train/video1_frame2450.json
Generating dataset from: images_masks/train/video2_frame1680.json
Generating dataset from: images_masks/train/video2_frame8640.json
Generating dataset from: images_m

<img src="labels_txt.PNG">

In [39]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes originales
!rm {images_dir}/test/*
!mv {images_dir}/test/coco/* {images_dir}/test
!rm -rf {images_dir}/test/coco 

!rm {images_dir}/train/*
!mv {images_dir}/train/coco/* {images_dir}/train
!rm -rf {images_dir}/train/coco 

!rm {images_dir}/val/*
!mv {images_dir}/val/coco/* {images_dir}/val
!rm -rf {images_dir}/val/coco 

!tree -d {images_dir}

rm: cannot remove 'images_masks/test/coco': Is a directory
rm: cannot remove 'images_masks/train/coco': Is a directory
rm: cannot remove 'images_masks/val/coco': Is a directory
images_masks
├── test
│   ├── JPEGImages
│   └── Visualization
├── train
│   ├── JPEGImages
│   └── Visualization
└── val
    ├── JPEGImages
    └── Visualization

9 directories


In [40]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 models/research/object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir={images_dir}/train \
--val_image_dir={images_dir}/val \
--test_image_dir={images_dir}/test \
--train_annotations_file={images_dir}/train/annotations.json \
--val_annotations_file={images_dir}/val/annotations.json \
--testdev_annotations_file={images_dir}/test/annotations.json \
--output_dir={training_dir}/tfrecord \
--include_masks=True

I1222 10:36:28.936357 140463654238016 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I1222 10:36:28.936712 140463654238016 create_coco_tf_record.py:412] 0 images are missing annotations.
I1222 10:36:28.936863 140463654238016 create_coco_tf_record.py:441] On image 0 of 241
I1222 10:36:44.573269 140463654238016 create_coco_tf_record.py:441] On image 100 of 241
I1222 10:37:01.575460 140463654238016 create_coco_tf_record.py:441] On image 200 of 241
I1222 10:37:08.215462 140463654238016 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I1222 10:37:08.221276 140463654238016 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I1222 10:37:08.221418 140463654238016 create_coco_tf_record.py:412] 0 images are missing annotations.
I1222 10:37:08.221460 140463654238016 create_coco_tf_record.py:441] On image 0 of 48
I1222 10:37:15.846401 140463654238016 create_coco_tf_record.py:466] Finished writing, 

# Entrenamiento y exportación del modelo

In [41]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training_masks
!python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir={training_dir}/ \
--pipeline_config_path={training_dir}/temp.config \
--num_train_steps=400000

W1222 10:37:29.867350 140296511833920 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 400000
I1222 10:37:29.867608 140296511833920 config_util.py:552] Maybe overwriting train_steps: 400000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1222 10:37:29.867669 140296511833920 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1222 10:37:29.867717 140296511833920 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1222 10:37:29.867767 140296511833920 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1222 10:37:29.867832 140296511833920 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/IPython/utils/_process_posix.py", line 162, in system
    res_idx = child.expect_list(patterns, self.read_timeout)
  File "/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/pexpect/spawnbase.py", line 372, in expect_list
    return exp.expect_loop(timeout)
  File "/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/pexpect/expect.py", line 169, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
  File "/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/pexpect/pty_spawn.py", line 500, in read_nonblocking
    if (timeout != 0) and select(timeout):
  File "/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/pexpect/pty_spawn.py", line 450, in select
    return select_ignore_interrupts([self.child_fd], [], [], timeout)[0]
  File "/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/pexpect/utils.py", line 143, in s

TypeError: object of type 'NoneType' has no len()

# para entrenarlo en la terminal copiar y pegar este fragmento

training_dir=training_masks_selected
python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir=$training_dir/ \
--pipeline_config_path=$training_dir/temp.config \
--num_train_steps=400000

In [50]:
# aquí nos vamos a fijar en los archivos model.ckpt y vamos a buscar el que tenga el número más alto, ya que este número
# indica el step del proceso de entrenamiento en el que se realizó el checkpoint del modelo
!dir {training_dir}

checkpoint
eval_0
events.out.tfevents.1603991149.bigdata-alineware
events.out.tfevents.1603991202.bigdata-alineware
events.out.tfevents.1603991247.bigdata-alineware
events.out.tfevents.1604050172.bigdata-alineware
events.out.tfevents.1604052895.bigdata-alineware
events.out.tfevents.1604068772.bigdata-alineware
graph.pbtxt
label_map.pbtxt
labels.txt
mask_rcnn_inception_v2_coco.config
model.ckpt-48666.data-00000-of-00001
model.ckpt-48666.index
model.ckpt-48666.meta
model.ckpt-48667.data-00000-of-00001
model.ckpt-48667.index
model.ckpt-48667.meta
model.ckpt-48668.data-00000-of-00001
model.ckpt-48668.index
model.ckpt-48668.meta
model.ckpt-48669.data-00000-of-00001
model.ckpt-48669.index
model.ckpt-48669.meta
model.ckpt-48670.data-00000-of-00001
model.ckpt-48670.index
model.ckpt-48670.meta
tfrecord


In [51]:
# por si no lo ves claro, corriendo esto te dirá qué debes poner en el script siguiente que generará el grafo de inferencia
import glob
import os
max_step=0
for file in glob.glob(training_dir+"/*"):
    if ("model.ckpt" in  file):
        step=int (file.split(os.sep)[1].split(".")[1].split("-")[1])
        max_step= step if step>max_step else max_step
        model_checkpoint_prefix="model.ckpt-{}".format(max_step)
print("En el campo trained_checkpoint debes poner lo siguiente: \n{}/model.ckpt-".format(training_dir)+str(max_step))

En el campo trained_checkpoint debes poner lo siguiente: 
training_masks_selected/model.ckpt-48670


In [55]:
# borramos el grafo de inferencia en caso de que queramos
!rm -r {inference_graph_dir}

In [52]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido del modelo entrenado y la fecha y la hora
!python3 models/research/object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path {training_dir}/mask_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix {training_dir}/{model_checkpoint_prefix} \
--output_directory {inference_graph_dir}

Instructions for updating:
Please use `layer.__call__` method instead.
W1103 08:14:00.560460 140209898223424 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I1103 08:14:01.412139 140209898223424 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1103 08:14:01.536967 140209898223424 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1103 08:14:01.537248 140209898223424 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1103 08:14:01.576214 140

In [53]:
!python3 scripts/tf_text_graph_mask_rcnn.py \
--input {inference_graph_dir}/frozen_inference_graph.pb \
--config {training_dir}/mask_rcnn_inception_v2_coco.config \
--output {inference_graph_dir}/graph.pbtxt

Number of classes: 5
Scales:            [0.25, 0.5, 1.0, 2.0]
Aspect ratios:     [0.5, 1.0, 2.0]
Width stride:      16.000000
Height stride:     16.000000
Features stride:   16.000000


# Generando predicciones en imágenes

In [54]:
!python3 scripts/predict_on_images.py \
-l via catenaria pk senal_fija senal_luminosa \
-m {inference_graph_dir} \
-i unlabeled \
-o predictions_masks_03_11_MVI_images \
-em 1 2 3 4 \
-eb 0 \
-t 0.6 \
-f jpg

output directory doesn't exist, creating it
predictions_masks_03_11_MVI_images/MVI_0098_frame6360_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0084_frame10960_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0084_frame9880_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0098_frame7360_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100_frame3480_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0101_frame12480_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100_frame3720_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100_frame7360_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100_frame3520_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0076_frame3760_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0084_frame5000_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0076_frame11520_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0098_frame6440_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100

KeyboardInterrupt: 